In [6]:
import argparse
import cv2
import numpy as np
import tensorflow as tf

In [12]:
path1=r'C:/Users/Legion/Documents/ParaAutonomia/Python/Curso/Deteccion/frozen_inference_graph_face.pb'

In [13]:
import numpy as np
import cv2
import tensorflow as tf

class TensoflowMobilNetSSDFaceDector():
    def __init__(self,
                 det_threshold=0.3,
                 model_path=path1):

        self.det_threshold = det_threshold
        self.detection_graph = tf.Graph()
        with self.detection_graph.as_default():
            od_graph_def = tf.GraphDef()
            with tf.gfile.GFile(model_path, 'rb') as fid:
                serialized_graph = fid.read()
                od_graph_def.ParseFromString(serialized_graph)
                tf.import_graph_def(od_graph_def, name='')

        with self.detection_graph.as_default():
            config = tf.ConfigProto()
            config.gpu_options.allow_growth = True
            self.sess = tf.Session(graph=self.detection_graph, config=config)

    def detect_face(self, image):

        h, w, c = image.shape

        image_np = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        image_np_expanded = np.expand_dims(image_np, axis=0)
        image_tensor = self.detection_graph.get_tensor_by_name(
            'image_tensor:0')

        boxes = self.detection_graph.get_tensor_by_name('detection_boxes:0')

        scores = self.detection_graph.get_tensor_by_name('detection_scores:0')
        classes = self.detection_graph.get_tensor_by_name(
            'detection_classes:0')
        num_detections = self.detection_graph.get_tensor_by_name(
            'num_detections:0')

        (boxes, scores, classes, num_detections) = self.sess.run(
            [boxes, scores, classes, num_detections],
            feed_dict={image_tensor: image_np_expanded})

        boxes = np.squeeze(boxes)
        scores = np.squeeze(scores)

        filtered_score_index = np.argwhere(
            scores >= self.det_threshold).flatten()
        selected_boxes = boxes[filtered_score_index]

        faces = np.array([[
            int(x1 * w),
            int(y1 * h),
            int(x2 * w),
            int(y2 * h),
        ] for y1, x1, y2, x2 in selected_boxes])

        return faces


In [14]:
def main():
        # Read image file using opencv
    cam = cv2.VideoCapture(0)
    
    
    face_detector = TensoflowMobilNetSSDFaceDector(
            det_threshold=0.3,model_path=path1)
    while True:
        ret,frame=cam.read()
        detected_face = face_detector.detect_face(frame)
    
        for face in detected_face:
            # Usage : cv2.rectangle( image_data,(xmin,ymin),(xmax,ymax),colour in BGR format : e.g (255,0,0) , line thickness = e.g 2 )
            cv2.rectangle(frame, (face[0], face[1]), (face[2], face[3]),
                          (255, 0, 0), 2)

        # Show result
        cv2.imshow('Face Detection', frame)
        k=cv2.waitKey(10) & 0xFF
        if k==ord('q'):
            break
    cv2.destroyAllWindows()
    cam.release()
    
    

In [15]:
main()

In [16]:
face_detector = TensoflowMobilNetSSDFaceDector(
            det_threshold=0.2,
            model_path=path1)

images=cv2.imread(r'C:/Users/Legion/Documents/ParaAutonomia/Python/Reconocimiento de patrones/tensorflow-face-object-detector-tutorial-master/Lucan-Parade-25.jpg')
detected_face = face_detector.detect_face(images)
for face in detected_face:
    # Usage : cv2.rectangle( image_data,(xmin,ymin),(xmax,ymax),colour in BGR format : e.g (255,0,0) , line thickness = e.g 2 )
    cv2.rectangle(images, (face[0], face[1]), (face[2], face[3]),
                          (255, 0, 0), 2)
cv2.imshow('Face Detection', images)
k=cv2.waitKey(5000)
images=cv2.imread(r'C:/Users/Legion/Documents/ParaAutonomia/Python/Reconocimiento de patrones/tensorflow-face-object-detector-tutorial-master/croud.jpg')
detected_face = face_detector.detect_face(images)
for face in detected_face:
    # Usage : cv2.rectangle( image_data,(xmin,ymin),(xmax,ymax),colour in BGR format : e.g (255,0,0) , line thickness = e.g 2 )
    cv2.rectangle(images, (face[0], face[1]), (face[2], face[3]),
                          (255, 0, 0), 2)
cv2.imshow('Face Detection', images)
k=cv2.waitKey(5000)
cv2.destroyAllWindows()